In [0]:
import numpy as np
import tensorflow as tf
import pickle

In [0]:
# downloading and loading images
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def gdrive():
    global drive
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

1


In [0]:
downloaded = drive.CreateFile({'id': "1hydbOxr3GLmm5_zaGOfshvml5fYY_P8a"})
downloaded.GetContentFile("train.pkl")
print "train"
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id': "1tXK20fHWydieLNhDiQo39ae0lfTxF3pa"})
downloaded.GetContentFile("validation.pkl")
print "validation"
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id': "1Xp4xV6yAqYpTMfOQIQ4wkDk8sZT7fHeN"})
downloaded.GetContentFile("test.pkl")
print "test"
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id': "1QDQxjVPxcDw7mrzMxep2OYCFkh8ElbSA"})
downloaded.GetContentFile("new.pkl")
print "new"
!ls

validation
test
new
adc.json  datalab  new.pkl  sample_data  test.pkl  validation.pkl


In [0]:
train = True
test = True
batch_size = 72
height = 101
width = 101
image_channels = 3
num_classes= 2

images = tf.placeholder(dtype = tf.float32,shape=[batch_size,height,width,image_channels])
masks = tf.placeholder(dtype=tf.float32 ,shape=[batch_size,height,width,num_classes])
train_status = tf.placeholder(dtype=tf.bool ,shape=[])


In [0]:
#encoder

#standardizing input images to 0 mean and 1 std dev
images = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images)
print images

with tf.variable_scope("encoder"):
    #conv0
    conv0_0 = tf.layers.conv2d(images,
                               filters=3,
                               kernel_size=2,
                               strides=(1, 1),
                               padding='valid',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv0_0 = tf.layers.batch_normalization(conv0_0,training = train_status)
    
    print conv0_0
    conv0_1 = tf.layers.conv2d(conv0_0,
                               filters=64,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv0_1 = tf.layers.batch_normalization(conv0_1,training = train_status)
    
    
    print conv0_1
    conv0_2 = tf.layers.conv2d(conv0_1,
                               filters=64,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv0_2 = tf.layers.batch_normalization(conv0_2,training = train_status)
    print conv0_2

    #pooling 1
    pool1 = tf.layers.max_pooling2d(conv0_2,
                                    pool_size=(2,2),
                                    strides=(2,2))
    print pool1

    #conv1
    conv1_0 = tf.layers.conv2d(pool1,
                               filters=128,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    
    conv1_0 = tf.layers.batch_normalization(conv1_0,training = train_status)
    print conv1_0
    
    conv1_1 = tf.layers.conv2d(conv1_0,
                               filters=128,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv1_1 = tf.layers.batch_normalization(conv1_1,training = train_status)
    
    print conv1_1

    #pooling 2
    pool2 = tf.layers.max_pooling2d(conv1_1,
                                    pool_size=(2,2),
                                    strides=(2,2))
    print pool2


    #conv2
    conv2_0 = tf.layers.conv2d(pool2,
                               filters=128,
                               kernel_size=2,
                               strides=(1, 1),
                               padding='valid',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv2_0 = tf.layers.batch_normalization(conv2_0,training = train_status)
    print conv2_0

    conv2_1 = tf.layers.conv2d(conv2_0,
                               filters=256,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv2_1 = tf.layers.batch_normalization(conv2_1,training = train_status)
    print conv2_1

    conv2_2 = tf.layers.conv2d(conv2_1,
                               filters=256,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv2_2 = tf.layers.batch_normalization(conv2_2,training = train_status)

    print conv2_2

    #pooling 3
    pool3 = tf.layers.max_pooling2d(conv2_2,
                                    pool_size=(2,2),
                                    strides=(2,2))
    print pool3
    #conv3
    conv3_0 = tf.layers.conv2d(pool3,
                               filters=512,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv3_0 = tf.layers.batch_normalization(conv3_0,training = train_status)
    print conv3_0
    
    conv3_1 = tf.layers.conv2d(conv3_0,
                               filters=512,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv3_1 = tf.layers.batch_normalization(conv3_1,training = train_status)
    print conv3_1

    #pooling 4
    pool4 = tf.layers.max_pooling2d(conv3_1,
                                    pool_size=(2,2),
                                    strides=(2,2))
    print pool4

    #conv4
    conv4_0 = tf.layers.conv2d(pool4,
                               filters=1024,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv4_0 = tf.layers.batch_normalization(conv4_0,training = train_status)
    
    print conv4_0
    conv4_1 = tf.layers.conv2d(conv4_0,
                               filters=1024,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    
    conv4_1 = tf.layers.batch_normalization(conv4_1,training = train_status)
    print conv4_1
    
    #pooling 5
    pool5 = tf.layers.max_pooling2d(conv4_1,
                                    pool_size=(2,2),
                                    strides=(2,2))
    print pool5   
    
    #convolution 5
    conv5_0 = tf.layers.conv2d(pool5,
                               filters=2048,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv5_0 = tf.layers.batch_normalization(conv5_0,training = train_status)
    print conv5_0
    
    conv5_1 = tf.layers.conv2d(conv5_0,
                               filters=2048,
                               kernel_size=3,
                               strides=(1, 1),
                               padding='same',
                               activation=tf.nn.relu,
                               kernel_initializer= tf.contrib.layers.xavier_initializer())
    conv5_1 = tf.layers.batch_normalization(conv5_1,training = train_status)
    print conv5_1
    


Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(72, 101, 101, 3), dtype=float32)
Tensor("encoder/batch_normalization/cond/Merge:0", shape=(72, 100, 100, 3), dtype=float32)
Tensor("encoder/batch_normalization_1/cond/Merge:0", shape=(72, 100, 100, 64), dtype=float32)
Tensor("encoder/batch_normalization_2/cond/Merge:0", shape=(72, 100, 100, 64), dtype=float32)
Tensor("encoder/max_pooling2d/MaxPool:0", shape=(72, 50, 50, 64), dtype=float32)
Tensor("encoder/batch_normalization_3/cond/Merge:0", shape=(72, 50, 50, 128), dtype=float32)
Tensor("encoder/batch_normalization_4/cond/Merge:0", shape=(72, 50, 50, 128), dtype=float32)
Tensor("encoder/max_pooling2d_1/MaxPool:0", shape=(72, 25, 25, 128), dtype=float32)
Tensor("encoder/batch_normalization_5/cond/Merge:0", shape=(72, 24, 24, 128), dtype=float32)
Tensor("encoder/batch_normalization_6/cond/Merge:0", shape=(72, 24, 24, 256), dtype=float32)
Tensor("encoder/batch_normalization_7/cond/Merge:0", shape=(72, 24, 24, 256), dtype=float32

In [0]:
#decoder
with tf.variable_scope("decoder"):    
    #deconvolution -1
    deconv = tf.layers.conv2d_transpose(conv5_1,
                                           filters = 1024,
                                           kernel_size = 2,
                                           strides=(2,2),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    
    deconv = tf.layers.batch_normalization(deconv,training = train_status)
    print deconv
    
    skip = tf.concat(values= [conv4_1,deconv],axis = 3)
    
    decoder_conv0 = tf.layers.conv2d(skip,
                                       filters=1024,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv0 = tf.layers.batch_normalization(decoder_conv0,training = train_status) 
    print decoder_conv0
    
    decoder_conv1 = tf.layers.conv2d(decoder_conv0,
                                       filters=1024,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv1 = tf.layers.batch_normalization(decoder_conv1,training = train_status) 
    print decoder_conv1
    
    #deconvolution 0
    deconv0_0 = tf.layers.conv2d_transpose(decoder_conv1,
                                           filters = 512,
                                           kernel_size = 2,
                                           strides=(2,2),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    deconv0_0 = tf.layers.batch_normalization(deconv0_0,training = train_status)
    print deconv0_0

    skip0 = tf.concat(values= [conv3_1,deconv0_0],axis = 3)

    print skip0

    decoder_conv0_0 = tf.layers.conv2d(skip0,
                                       filters=512,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv0_0 = tf.layers.batch_normalization(decoder_conv0_0,training = train_status)

    print decoder_conv0_0

    decoder_conv0_1 = tf.layers.conv2d(decoder_conv0_0,
                                       filters=512,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv0_1 = tf.layers.batch_normalization(decoder_conv0_1,training = train_status)
    print decoder_conv0_1

    
    #deconvolution 1
    deconv1_0 = tf.layers.conv2d_transpose(decoder_conv0_1,
                                           filters = 256,
                                           kernel_size = 2,
                                           strides=(2,2),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    
    deconv1_0 = tf.layers.batch_normalization(deconv1_0,training = train_status)
    print deconv1_0

    skip1 = tf.concat(values= [conv2_2,deconv1_0],axis = 3)

    print skip1

    decoder_conv1_0 = tf.layers.conv2d(skip1,
                                       filters=256,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv1_0 = tf.layers.batch_normalization(decoder_conv1_0,training = train_status)
    print decoder_conv1_0

    decoder_conv1_1 = tf.layers.conv2d(decoder_conv1_0,
                                       filters=256,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv1_1 = tf.layers.batch_normalization(decoder_conv1_1,training = train_status)
    print decoder_conv1_1

    deconv1_1 = tf.layers.conv2d_transpose(decoder_conv1_1,
                                           filters = 256,
                                           kernel_size = 2,
                                           strides=(1,1),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv1_1 = tf.layers.batch_normalization(deconv1_1,training = train_status)
    print deconv1_1

    #deconvolution 2
    deconv2_0 = tf.layers.conv2d_transpose(deconv1_1,
                                           filters = 128,
                                           kernel_size = 2,
                                           strides=(2,2),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    deconv2_0 = tf.layers.batch_normalization(deconv2_0,training = train_status)
    print deconv2_0

    skip2 = tf.concat(values= [conv1_1,deconv2_0],axis = 3)

    print skip2

    decoder_conv2_0 = tf.layers.conv2d(skip2,
                                       filters=128,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv2_0 = tf.layers.batch_normalization(decoder_conv2_0,training = train_status)

    print decoder_conv2_0

    decoder_conv2_1 = tf.layers.conv2d(decoder_conv2_0,
                                       filters=128,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv2_1 = tf.layers.batch_normalization(decoder_conv2_1,training = train_status)
    
    print decoder_conv2_1

    #deconvolution 2
    deconv3_0 = tf.layers.conv2d_transpose(decoder_conv2_1,
                                           filters = 64,
                                           kernel_size = 2,
                                           strides=(2,2),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    deconv3_0 = tf.layers.batch_normalization(deconv3_0,training = train_status)

    print deconv3_0

    skip3 = tf.concat(values= [conv0_2,deconv3_0],axis = 3)

    print skip3

    decoder_conv3_0 = tf.layers.conv2d(skip3,
                                       filters=64,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv3_0 = tf.layers.batch_normalization(decoder_conv3_0,training = train_status)

    print decoder_conv3_0

    decoder_conv3_1 = tf.layers.conv2d(decoder_conv3_0,
                                       filters=64,
                                       kernel_size=3,
                                       strides=(1, 1),
                                       padding='same',
                                       activation=tf.nn.relu,
                                       kernel_initializer= tf.contrib.layers.xavier_initializer())
    decoder_conv3_1 = tf.layers.batch_normalization(decoder_conv3_1,training = train_status)

    print decoder_conv3_1

    deconv3_1 = tf.layers.conv2d_transpose(decoder_conv3_1,
                                           filters = 64,
                                           kernel_size = 2,
                                           strides=(1,1),
                                           padding='valid',
                                           activation=tf.nn.relu,
                                           kernel_initializer= tf.contrib.layers.xavier_initializer())
    deconv3_1 = tf.layers.batch_normalization(deconv3_1,training = train_status)

    print deconv3_1

Tensor("decoder/batch_normalization/cond/Merge:0", shape=(72, 6, 6, 1024), dtype=float32)
Tensor("decoder/batch_normalization_1/cond/Merge:0", shape=(72, 6, 6, 1024), dtype=float32)
Tensor("decoder/batch_normalization_2/cond/Merge:0", shape=(72, 6, 6, 1024), dtype=float32)
Tensor("decoder/batch_normalization_3/cond/Merge:0", shape=(72, 12, 12, 512), dtype=float32)
Tensor("decoder/concat_1:0", shape=(72, 12, 12, 1024), dtype=float32)
Tensor("decoder/batch_normalization_4/cond/Merge:0", shape=(72, 12, 12, 512), dtype=float32)
Tensor("decoder/batch_normalization_5/cond/Merge:0", shape=(72, 12, 12, 512), dtype=float32)
Tensor("decoder/batch_normalization_6/cond/Merge:0", shape=(72, 24, 24, 256), dtype=float32)
Tensor("decoder/concat_2:0", shape=(72, 24, 24, 512), dtype=float32)
Tensor("decoder/batch_normalization_7/cond/Merge:0", shape=(72, 24, 24, 256), dtype=float32)
Tensor("decoder/batch_normalization_8/cond/Merge:0", shape=(72, 24, 24, 256), dtype=float32)
Tensor("decoder/conv2d_transp

In [0]:
with tf.variable_scope("prediction"):
    pred_mask = tf.layers.conv2d(deconv3_1,
                                 filters=2,
                                 kernel_size=1,
                                 strides=(1, 1),
                                 padding='same',
                                 activation=None,    #loss function applies softmax internally
                                 kernel_initializer= tf.contrib.layers.xavier_initializer()) 
    print pred_mask

Tensor("prediction/conv2d/BiasAdd:0", shape=(72, 101, 101, 2), dtype=float32)


In [0]:
with tf.variable_scope("loss"):
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = masks, logits = pred_mask)
    loss = tf.reduce_mean(loss)

In [0]:
learning_rate = 1e-3
minimize = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [0]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

def save_model(sess):
    !mkdir saved_model
    saver = tf.train.Saver()
    path = saver.save(sess,"./saved_model/model.ckpt")
    print "Model saved @ " + path
    return path
  
def restore_model(sess):
    saver = tf.train.Saver()
    saver.restore(sess, "./saved_model/model.ckpt")
    
def restore_model_from_drive(sess,file_id):
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile("saved_model.tar.gz")
    !tar -xzvf saved_model.tar.gz
    restore_model(sess)

  
  
def upload_to_google_drive(filename,folder_id):
    uploaded = False
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
    try:
        for file1 in file_list:
            if (file1['title'] == filename) and (not uploaded):
                file1.Delete()
                f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id, }]})
                f.SetContentFile(filename)
                f.Upload()
                uploaded = True
        if not uploaded:
            f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id, }]})
            f.SetContentFile(filename)
            f.Upload()
            uploaded = True
    except:
        print "Failed"
  
    return f.get("id")

def get_one_hot_masks(masks):
    temp = (np.invert(masks.astype(dtype=bool)).astype(dtype = int))
    masks=np.expand_dims(masks,axis=3)
    temp =np.expand_dims(temp,axis=3)
    return np.concatenate([temp,masks],axis = 3)



In [0]:
def get_ious(test_images,true_masks, batch_size):
    no_of_batches = len(true_masks)/batch_size
    ptr = 0
    ious = []
    for i in range(no_of_batches):
        x = test_images[ptr:ptr+batch_size]
        y = get_one_hot_masks(true_masks[ptr:ptr+batch_size])
        prediction = sess.run(pred_mask,{images:x,masks:y,train_status: True})
        prediction = np.argmax(prediction,axis=3)
        true_mask_bool = true_masks[ptr:ptr+batch_size].astype(dtype=bool)
        pred_mask_bool = prediction.astype(dtype=bool)

        for i in range(batch_size):
            temp = (true_mask_bool[i]*pred_mask_bool[i])
            intersection = np.sum(temp)
            temp = (true_mask_bool[i]+pred_mask_bool[i])
            union = np.sum(temp)
            
            if union:
                ious.append(float(intersection)/union)
            else:
                ious.append(1.0)
                
        ptr += batch_size
   
    return np.array(ious)

def train_model(train_images,train_masks,no_of_batches):
    total = 0
    ptr = 0
    for j in range(no_of_batches):
        total+=1
        if j%10 == 0:
            print total
        x = train_images[ptr:ptr+batch_size]
        y = get_one_hot_masks(train_masks[ptr:ptr+batch_size])
        sess.run(minimize,{images: x, masks: y,train_status: True})

        ptr+= batch_size
    return total

In [0]:
if train:
    folder_id = "1wxr6L8DIiFxzNC752-b4CgY55mlkVOsc"  #kaggle unet folder in google drive
    filename = "saved_model.tar.gz"
    curr_iou =  0
    saved_iou= 0
    epoch = 1000
    with open("validation.pkl","r") as f:
        validation_images, validation_masks= pickle.load(f)
        
    with open(str(1)+".pkl","r") as f:
        train_images , train_masks = pickle.load(f)
    
    for i in range(epoch):
        gdrive()
        print "epoch------------>>>  "+ str(i)
        batches = 0
        x = None
        y = None
        for k in range(1):
            no_of_batches = len(train_images)/batch_size
            
            batches = train_model(train_images,train_masks,no_of_batches)
            print batches
            x = train_images[:batch_size]
            y = get_one_hot_masks(train_masks[:batch_size])
            curr_iou = np.mean(get_ious(validation_images,validation_masks,batch_size))
            print "loss: " , sess.run(loss,{images: x, masks: y,train_status: True})
            print "mean iou = " + str(curr_iou)
            if curr_iou > saved_iou:
                save_model(sess)
                !tar -czvf saved_model.tar.gz saved_model
                print upload_to_google_drive(filename,folder_id)
                saved_iou= curr_iou
             

epoch------------>>>  0
1
11
21
31
41
50
loss:  0.40510008
mean iou = 0.27843074609684654
Model saved @ ./saved_model/model.ckpt
saved_model/
saved_model/checkpoint
saved_model/model.ckpt.data-00000-of-00001
saved_model/model.ckpt.index
saved_model/model.ckpt.meta
1GllrK0-VOAiuPFkpKKvMT-8KybRAuqOO
1
11
21
31
41
50
loss:  0.33843273
mean iou = 0.418370932909938
mkdir: cannot create directory ‘saved_model’: File exists
Model saved @ ./saved_model/model.ckpt
saved_model/
saved_model/checkpoint
saved_model/model.ckpt.data-00000-of-00001
saved_model/model.ckpt.index
saved_model/model.ckpt.meta
1NzBNDvR6TL9eMSznPcFJF6QEOsOHa5l0
epoch------------>>>  1
1
11
21
31
41
50
loss:  0.28482533
mean iou = 0.5910858495183885
mkdir: cannot create directory ‘saved_model’: File exists
Model saved @ ./saved_model/model.ckpt
saved_model/
saved_model/checkpoint
saved_model/model.ckpt.data-00000-of-00001
saved_model/model.ckpt.index
saved_model/model.ckpt.meta
1JWcQ8bnZ1Ur5JTbrdZ4ixi9XPcG23ngk
1
11
21
31
41


In [0]:

def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [0]:
import csv
def add_to_submission(runs,image_ids):
    with open('submission.csv', 'a') as csvFile:
        writer=csv.writer(csvFile)
        for i in range(len(image_ids)):
            row = [image_ids[i],runs[i]]
            writer.writerow(row)
    csvFile.close()
            
if test:
    file_id ="1S2NP8laV8soVXNFHi-ff-Fcs5WXa2FH5"
    restore_model_from_drive(sess,file_id)
    with open("new.pkl","r") as f:
        new_images, image_ids = pickle.load(f)
    print image_ids[0]
    print new_images.shape
    predicted_masks = []
    
#     normalize new images. 3 channels seperately
    img_a = new_images[:, :, :, 0]
    img_b = new_images[:, :, :, 1]
    img_c = new_images[:, :, :, 2]

    # normalizing per channel data:
    img_a = (img_a - np.min(img_a)).astype(dtype=float) / (np.max(img_a) - np.min(img_a))
    img_b = (img_b - np.min(img_b)).astype(dtype=float) / (np.max(img_b) - np.min(img_b))
    img_c = (img_c - np.min(img_c)).astype(dtype=float) / (np.max(img_c) - np.min(img_c))

    # putting the 3 channels back together:
    new_images = np.empty(new_images.shape, dtype=np.float32)
    new_images[:, :, :, 0] = img_a
    new_images[:, :, :, 1] = img_b
    new_images[:, :, :, 2] = img_c

    del img_a,img_b,img_c

    

    header = ["id","rle_mask"]
    
    with open('submission.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(header)

    csvFile.close()
    
    #get masks for the new images
    ptr = 0 
    no_of_batches = new_images.shape[0]/batch_size
    
    
    for i in range(no_of_batches):
        x = new_images[ptr:ptr+batch_size]        
        predicted_masks = np.argmax(sess.run(pred_mask,{images:x,train_status: True}),axis=3).astype(dtype=int)
        runs = []
        for i in predicted_masks:
            runs.append(RLenc(i))
        print i
        add_to_submission(runs,image_ids[ptr:ptr+batch_size])
        ptr+=batch_size

        
        
    #upload to google drive
    folder_id = "1wxr6L8DIiFxzNC752-b4CgY55mlkVOsc" 
    upload_to_google_drive("submission.csv",folder_id)

saved_model/
saved_model/checkpoint
saved_model/model.ckpt.data-00000-of-00001
saved_model/model.ckpt.index
saved_model/model.ckpt.meta
INFO:tensorflow:Restoring parameters from ./saved_model/model.ckpt
e45d4eb76a
(18000, 101, 101, 3)
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
add